In [ ]:
# Matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
# Numpy
import numpy as np
# Pandas
import pandas as pd
# Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchmetrics.classification import Accuracy
from model import ResNetLSTM as model

In [ ]:
import pickle
from sklearn.model_selection import train_test_split

In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, npy_file_paths, labels):
        """
        Args:
            npy_file_paths (list of str): List of file paths for .npy files containing the sequences.
            labels (list): List of labels corresponding to each sequence.
        """
        # Load the sequences and labels
        self.data = [torch.tensor(np.load(file_path)) for file_path in npy_file_paths]
        self.labels = torch.tensor(labels)  # Convert the labels to a tensor

    def __len__(self):
        # Dataset contains as many samples as the number of npy files
        return len(self.data)

    def __getitem__(self, idx):
        # Return the sequence data and its corresponding label
        return self.data[idx], self.labels[idx]

# Example usage
# Add paths to your .npy files containing sequences, using for/while loop
npy_file_paths = [] # ['arabic0.npy', 'arabic1.npy', 'chinese0.npy'] 

# Corresponding labels for the sequences - depending on language (for loop)
labels = [0, 0, 1]
# adjust fc layer in models.py based on total number of languages

dataset = SequenceDataset(npy_file_paths, labels)

# dataloader = DataLoader(dataset, batch_size=64)

In [ ]:
def split_data(dataset):
    # Split dataset into train, validation, and test sets
    train_data, temp_data = train_test_split(dataset, test_size=0.3, shuffle=True)
    valid_data, test_data = train_test_split(temp_data, test_size=0.5, shuffle=True)

    # Define batch size for training, full batch for validation and testing
    batch_size = 128

    # Create DataLoader objects
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)  # Mini-batch for training
    valid_loader = DataLoader(valid_data, batch_size=len(valid_data), shuffle=False)  # Full batch for validation
    test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)  # Full batch for testing

    return train_loader, valid_loader, test_loader

In [ ]:
train_loader, valid_loader, test_loader = split_data(dataset)

In [ ]:
class Trainer:
    def __init__(self, model, train_loader, valid_loader, test_loader, num_epochs=10, gradual_unfreezing=False, unfreeze_epochs=[]):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.test_loader = test_loader
        self.num_epochs = num_epochs
        self.gradual_unfreezing = gradual_unfreezing
        self.unfreeze_epochs = (
            [] if unfreeze_epochs is None or not gradual_unfreezing 
            else sorted(unfreeze_epochs, reverse=True)  # Unfreeze last layers first
        )
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.01, betas=(0.9, 0.999), eps=1e-08)
        self.optimizer.zero_grad()
        self.accuracy_metric = Accuracy(task="multiclass", num_classes=self.model.num_classes).to(self.device)
        self.best_val_loss = float('inf')
        self.best_model_state = None
        self.best_epoch = 0
        self.l2_lambda = 0.001
        self.layer_groups = self._group_layers()
        self._initialize_requires_grad()

    def _group_layers(self):
        layer_dict = OrderedDict()
        for name, param in self.model.named_parameters():
            layer_name = name.split(".")[0]  # Extract top-level layer name
            if layer_name not in layer_dict:
                layer_dict[layer_name] = []
            layer_dict[layer_name].append(param)
        return list(layer_dict.items())  # List of (layer_name, parameters) tuples

    def _initialize_requires_grad(self):
        if not self.gradual_unfreezing:  # If gradual_unfreezing is False, make all parameters trainable
            for param in self.model.parameters():
                param.requires_grad = True
        else:
            for layer_name, params in self.layer_groups:
                for param in params:
                    param.requires_grad = False
            self._unfreeze_layer() #unfreeze fc layer

    def _unfreeze_layer_by_epoch(self, epoch):
        if epoch in self.unfreeze_epochs:
            self._unfreeze_layer()
    
    def _unfreeze_layer(self):
        if self.layer_groups:
            layer_name, params = self.layer_groups.pop()  # Unfreeze last remaining layer
            for param in params:
                param.requires_grad = True

    def train(self):
        for epoch in range(self.num_epochs):
            self.model.train()
            self._unfreeze_layer_by_epoch(epoch)
            for inputs, targets in self.train_loader:
                inputs, targets = inputs.float().to(self.device), targets.to(self.device)
                pred = self.model(inputs)
                loss = self.criterion(pred, targets)
                
                # L2 Regularization
                l2_norm = sum(p.pow(2).sum() for p in self.model.parameters())
                loss += self.l2_lambda * l2_norm
                
                self.accuracy_metric.update(pred, targets)
                accuracy = self.accuracy_metric.compute()
                self.accuracy_metric.reset()
                
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
            
            val_loss, val_accuracy = self.validate()
            
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.best_model_state = self.model.state_dict()
                self.best_epoch = epoch + 1
                self._save_best_model()
            
            print(f'Epoch [{epoch+1}/{self.num_epochs}], Training Loss: {loss.item():.4f}, Training Accuracy: {accuracy.item():.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')
        
        self._load_best_model()
        self.test()

    def validate(self):
        self.model.eval()
        with torch.no_grad():
            inputs, targets = next(iter(self.valid_loader))
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            pred = self.model(inputs)
            loss = self.criterion(pred, targets).item()
            self.accuracy_metric.update(pred, targets)
            accuracy = self.accuracy_metric.compute()
            self.accuracy_metric.reset()
        return loss, accuracy

    def test(self):
        self.model.eval()
        with torch.no_grad():
            inputs, targets = next(iter(self.test_loader))
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            pred = self.model(inputs)
            loss = self.criterion(pred, targets).item()
            self.accuracy_metric.update(pred, targets)
            accuracy = self.accuracy_metric.compute()
            self.accuracy_metric.reset()
        print(f'Final Test Loss: {loss:.4f}, Final Test Accuracy: {accuracy:.4f}')

    def _save_best_model(self):
        with open("best_model.pkl", "wb") as f:
            pickle.dump({"model_state": self.best_model_state, "epoch": self.best_epoch, "val_loss": self.best_val_loss}, f)

    def _load_best_model(self):
        with open("best_model.pkl", "rb") as f:
            saved_data = pickle.load(f)
            self.model.load_state_dict(saved_data["model_state"])
            print(f"Best Model Achieved at Epoch: {saved_data['epoch']} with Validation Loss: {saved_data['val_loss']:.4f}")

In [ ]:
trainer = Trainer(ResNet50(num_classes=3), train_loader, valid_loader, test_loader)
trainer.train()